In [1]:
import numpy as np

In [2]:
A = np.array([[2., 7., -8., 6.],
            [4., 4., 0., -7.],
            [-1., -3., 6., 3.],
            [9., -7., -2., -8.]])

In [3]:
f = np.array([-39., 41., 4., 113.])

### LU - разложение

In [4]:
def LU_decomposition(a):
    # Представим матрицу А в виде произведения двух матриц, A = LU, где
    # L - нижнетреугольная матрица с единицами на главной диагонали
    # U - верхнетреугольная матрица
    l = np.zeros_like(a)
    u = np.zeros_like(a)
    
    n = len(a)
    
    for j in range(n):
        u[0][j] = a[0][j]
        l[j][0] = float(a[j][0]) / float(u[0][0])
        
    for i in range(1,  n):
        for j in range(i, n):
            cur_sum = 0
            for k in range(i):
                cur_sum += l[i][k] * u[k][j]
            u[i][j] = a[i][j] - cur_sum
        for j in range(i, n):
            cur_sum = 0
            for k in range(i):
                cur_sum += l[j][k] * u[k][i]
            l[j][i] = (a[j][i] - cur_sum) / u[i][i]
    return l, u

In [5]:
A

array([[ 2.,  7., -8.,  6.],
       [ 4.,  4.,  0., -7.],
       [-1., -3.,  6.,  3.],
       [ 9., -7., -2., -8.]])

In [6]:
l, u = LU_decomposition(A)

In [7]:
l @ u

array([[ 2.,  7., -8.,  6.],
       [ 4.,  4.,  0., -7.],
       [-1., -3.,  6.,  3.],
       [ 9., -7., -2., -8.]])

In [8]:
l

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 2.        ,  1.        ,  0.        ,  0.        ],
       [-0.5       , -0.05      ,  1.        ,  0.        ],
       [ 4.5       ,  3.85      , -9.85714286,  1.        ]])

In [9]:
u

array([[  2.        ,   7.        ,  -8.        ,   6.        ],
       [  0.        , -10.        ,  16.        , -19.        ],
       [  0.        ,   0.        ,   2.8       ,   5.05      ],
       [  0.        ,   0.        ,   0.        ,  87.92857143]])

In [10]:
def solution_with_LU(l, u, f):
    
    n = len(l)
    y = np.zeros(n)
    for i in range(n):
        cur_sum = 0
        for k in range(i):
            cur_sum += l[i][k] * y[k]
        y[i] = f[i] - cur_sum
    
    x = np.zeros(n)
    
    for i in range(n):
        cur_sum = 0
        for k in range(i):
            cur_sum += u[n - 1 - i][n - 1 - k] * x[n - 1 - k]
        x[n - 1 - i] = (y[n - 1 - i] - cur_sum) / u[n - 1 - i][n - 1 - i]
    return x

In [11]:
solution_with_LU(l, u, f)

array([ 8., -3.,  2., -3.])

### Метод простых итераций

In [12]:
# Выберем tau = 2/(lambda_min + lambda_max)
lambds, q = np.linalg.eig(A)
tau = 2/(np.max(lambds) + np.min(lambds))
tau

(-0.2197263533843342-0.047494776136670254j)

In [13]:
# Возьмем tau равным
tau = 0.0234

In [14]:
def metric(x, y):
    return np.sqrt(np.sum((x - y)**2))

In [19]:
def mpi(a, f, tau=0.0234, eps = 10e-6):
    E = np.diag(np.ones(len(a)))
    F = tau * f
    R = E - tau * a
    # начальное приближение
    u_k = np.zeros(len(a))
    prev = u_k
    metr = 10
    i = 0
    while (i < 1000 and metr > 10e-6): 
        i += 1
        u_k = R @ prev + F
        metr = metric(u_k, prev)
        prev = u_k
        print(f'metrica = {metr}, sequences = {prev}')
    return u_k

In [41]:
tau = 0.0234 #0.000001
E = np.diag(np.ones(len(A)))
R = E - tau * A
print(R)
np.max(np.sum(np.abs(R), axis = 1))

[[ 0.7894  0.1638  0.0468  0.1872]
 [-0.0936  0.9064  0.      0.1638]
 [ 0.0234  0.0702  0.8596 -0.0702]
 [-0.0468 -0.1638  0.1872  0.8596]]


1.2574

In [39]:
# Для сходимости МПИ
# поменяем местами строки матрицы
A = np.array([[9., -7., -2., -8.],
              [4., 4., 0., -7.],
              [-1., -3., 6., 3.],             
              [2., 7., -8., 6.]])

f = np.array([113., 41., 4., -39.])

In [40]:
mpi(A, f)

metrica = 2.9586891894891565, sequences = [ 2.6442  0.9594  0.0936 -0.9126]
metrica = 2.390495733132579, sequences = [ 4.72222296  1.43201916  0.36734724 -1.96044732]
metrica = 1.886784424997995, sequences = [ 6.25668366  1.49426083  0.75802285 -2.98459789]
metrica = 1.4679494652973433, sequences = [ 7.30474475  1.23929529  1.20601872 -3.87383118]
metrica = 1.1465228200862942, sequences = [ 7.94482255  0.76443959  1.6601662  -4.5616372 ]
metrica = 0.9234571770498521, sequences = [ 8.26481542  0.16145648  2.0804783  -5.02003313]
metrica = 0.7838936412611839, sequences = [ 8.35250805 -0.49012399  2.4391164  -5.25159487]
metrica = 0.6994755283008256, sequences = [ 8.28843963 -1.12685438  2.7199684  -5.28088343]
metrica = 0.6406491617895481, sequences = [ 8.14122864 -1.70278747  2.91724717 -5.14618954]
metrica = 0.5876951779296306, sequences = [ 7.96512979 -2.18897141  3.03349724 -4.89224877]
metrica = 0.532248387313433, sequences = [ 7.79945864 -2.57157018  3.07734833 -4.56432092]
metrica

array([ 7.99994517, -2.99998968,  1.9999928 , -3.00003079])

### Метод Зейделя

In [0]:
from math import sqrt
import numpy as np

def seidel(A, b, eps):
    
    n = len(A)
    x = [.0 for i in range(n)]

    converge = False
    while not converge:
        x_new = np.copy(x)
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]

        converge = sqrt(sum((x_new[i] - x[i]) ** 2 for i in range(n))) <= eps
        x = x_new

    return x

In [0]:
# Для сходимости метода Зейделя
# поменяем местами строки матрицы
A = np.array([[9., -7., -2., -8.],
              [2., 7., -8., 6.],
              [-1., -3., 6., 3.],
              [4., 4., 0., -7.]])

f = np.array([113., -39., 4., 41.])

In [23]:
seidel(A, f, eps=10e-6)

array([ 7.99999624, -2.99999441,  2.00000449, -2.99999895])